Word2Vec + Decision Tree Regressor

In [1]:
# imports
import pandas as pd
import tensorflow as tf
import preprocess_kgptalkie as ps
import nltk

In [2]:
# installations
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\helen\appdata\local\temp\pip-req-build-njtmk3wp
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 9ca68d37027af9f6a30d54640347ce3b2e2694b3
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7692 sha256=64761b72307bba0987d0d65dd8e4d30786b404a5e5c86db2320cfa92e00edc9b
  Stored in directory: C:\Users\helen\AppData\Local\Temp\pip-ephem-wheel-cache-agpns529\wheels\cd\26\af\103cfac199bd773bfc95ed85a52fe9095e4d902ed9ac3b0c97
Successfully built preprocess-kgptalkie
  Attempting uninstall: preprocess-kgptalkie
    Found existing installation: preprocess-kgptalkie 0.1.3
    Uninstalling preprocess-kgptalkie-0.1.3:
      Successfully uninstalled preprocess-kgptalkie-0.1.3


  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git 'C:\Users\helen\AppData\Local\Temp\pip-req-build-njtmk3wp'


Dataset Preprocessing
* total data count: 19704
    * train: 14789 
    * test: 4915

In [74]:
# training data
train_dataset = pd.read_csv('drugsComTrain_raw.csv')
train_dataset.dropna(inplace=True)

antidepressants_list = ['Escitalopram', 'Vilazodone', 'Lexapro', 'Trintellix', 'Paroxetine', 'Sertraline', 
                        'Citalopram', 'Fluoxetine', 'Zoloft', 'Celexa', 'Prozac', 'Venlafaxine', 'Pristiq', 
                        'Desvenlafaxine', 'Duloxetine', 'Cymbalta', 'Viibryd', 'Effexor', 'Effexor XR', 
                        'Amitriptyline', 'Amoxapine', 'Desipramine', 'Doxepin', 'Imipramine', 'Nortriptyline', 
                        'Protriptyline', 'Trimipramine', 'Isocarboxazid', 'Phenelzine', 'Selegiline', 
                        'Tranylcypromine']

train_dataset = train_dataset[train_dataset.drugName.isin(antidepressants_list)]
train_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
17,219869,Amitriptyline,ibromyalgia,"""I&#039;ve been taking amitriptyline since Jan...",9,15-Mar-17,39
31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,7-May-11,3
44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,27-Apr-16,3
60,131704,Effexor XR,Anxiety,"""Was on this med for 5 years. Worked fine but ...",6,27-Dec-16,23
67,131909,Effexor XR,Depression,"""This medicine saved my life. I was at my wits...",10,20-Jun-13,166
...,...,...,...,...,...,...,...
161253,131713,Effexor XR,Generalized Anxiety Disorde,"""I started to take this med one week ago for m...",5,16-Nov-16,19
161260,168041,Imipramine,Night Terrors,"""I had panic attacks (Night Terrors) for over ...",10,2-Jan-10,60
161270,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89
161284,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25


In [75]:
# testing data
test_dataset = pd.read_csv('drugsComTest_raw.csv')
test_dataset.dropna(inplace=True)

antidepressants_list = ['Escitalopram', 'Vilazodone', 'Lexapro', 'Trintellix', 'Paroxetine', 'Sertraline', 
                        'Citalopram', 'Fluoxetine', 'Zoloft', 'Celexa', 'Prozac', 'Venlafaxine', 'Pristiq', 
                        'Desvenlafaxine', 'Duloxetine', 'Cymbalta', 'Viibryd', 'Effexor', 'Effexor XR', 
                        'Amitriptyline', 'Amoxapine', 'Desipramine', 'Doxepin', 'Imipramine', 'Nortriptyline', 
                        'Protriptyline', 'Trimipramine', 'Isocarboxazid', 'Phenelzine', 'Selegiline', 
                        'Tranylcypromine']

test_dataset = test_dataset[test_dataset.drugName.isin(antidepressants_list)]
test_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on...",9,21-Apr-09,32
10,196802,Paroxetine,Hot Flashes,"""Holy Hell is exactly how I feel. I had been t...",1,22-Feb-17,17
36,178004,Duloxetine,Pain,"""Been on 30mg Cymbalta for 2 weeks. Started ge...",9,27-Mar-12,32
38,141462,Escitalopram,Depression,"""I am a 22 year old female college student. I ...",9,29-Apr-14,32
63,131523,Effexor XR,Anxiety,"""I was prescribed this for onset of anxiety an...",2,27-Aug-17,13
...,...,...,...,...,...,...,...
53692,141481,Escitalopram,Generalized Anxiety Disorde,"""I was on Lexapro for a year and decided to co...",9,6-Nov-13,14
53702,201249,Zoloft,Post Traumatic Stress Disorde,"""Didn&#039;t do much for me except killed my s...",2,20-Jul-16,14
53710,64806,Sertraline,Obsessive Compulsive Disorde,"""I&#039;m 14 years old with OCD. I tried Proza...",9,27-Jul-15,82
53732,28754,Lexapro,Depression,"""I&#039;m a 19 year old girl and I&#039;ve bee...",7,20-Aug-10,26


In [59]:
# preprocessing: lowercase, punctuation removal, tokenization, stop word removal, stemming, lemmatization
# lowercase
train_dataset['review'] = train_dataset['review'].apply(lambda x: str(x).lower())

# punctuation removal
train_dataset['review'] = train_dataset['review'].apply(lambda x: ps.remove_special_chars(x))

In [50]:
train_dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
17,219869,Amitriptyline,ibromyalgia,i039ve been taking amitriptyline since january...,9,15-Mar-17,39
31,96233,Sertraline,Depression,1 week on zoloft for anxiety and mood swings i...,8,7-May-11,3
44,121333,Venlafaxine,Depression,my gp started me on venlafaxine yesterday to h...,4,27-Apr-16,3
60,131704,Effexor XR,Anxiety,was on this med for 5 years worked fine but no...,6,27-Dec-16,23
67,131909,Effexor XR,Depression,this medicine saved my life i was at my wits e...,10,20-Jun-13,166
...,...,...,...,...,...,...,...
161253,131713,Effexor XR,Generalized Anxiety Disorde,i started to take this med one week ago for my...,5,16-Nov-16,19
161260,168041,Imipramine,Night Terrors,i had panic attacks night terrors for over a y...,10,2-Jan-10,60
161270,183202,Cymbalta,Anxiety,i have been taking cymbalta for 15 months now ...,9,10-Jun-13,89
161284,121154,Venlafaxine,Panic Disorde,had panic attacks and social anxiety starting ...,9,10-Nov-16,25


In [99]:
from tqdm import tqdm # shows progress + run time

for index, row in tqdm(train_dataset.iterrows()):
    review_text = row['review']
    review_id = row['uniqueID']
    
    filename = 'C:/Users/helen/OneDrive/Documents/GitHub/drugreviews/reviews_corpus/{}.txt'.format(review_id)
    
    f = open(filename, 'w', encoding='utf-8') # w opens the file for writing, u
    f.write(review_text)
    f.close()

14789it [00:14, 986.95it/s] 


In [100]:
review_id

45410